In [1]:
import nb_setup 
import importlib
importlib.reload(nb_setup)
nb_setup.init()

Added project root to sys.path: /Users/falcon/Developments/act/backtesting-playground/trading
Changed current working directory to: /Users/falcon/Developments/act/backtesting-playground/trading
Initialized project with base directory: /Users/falcon/Developments/act/backtesting-playground/trading


In [2]:
import models.market as market
import importlib
importlib.reload(market)

market = market.Market.from_json_file()
df = market.to_df()
df.head()

,maxLeverage,name,onlyIsolated,szDecimals,dayNtlVlm,funding,impactPxs,markPx,midPx,openInterest,oraclePx,premium,prevDayPx
0,50,BTC,False,5,3.986197e+08,0.000013,"[66056.4, 66064.0]",66063.00000,66063.50000,1.703480e+03,66082.00000,-0.000272,65105.00000
1,50,ETH,False,4,3.622029e+08,0.000013,"[3183.71, 3184.13]",3183.80000,3183.85000,3.174533e+04,3184.80000,-0.000210,3160.30000
2,20,ATOM,False,2,1.982419e+06,-0.000018,"[8.8203, 8.8266]",8.82240,8.82160,9.740096e+04,8.83110,-0.000510,8.69400
3,20,MATIC,False,1,7.079440e+06,0.000013,"[0.7375, 0.73773]",0.73747,0.73762,2.684614e+06,0.73765,0.000000,0.71934
4,20,DYDX,False,1,1.551456e+06,0.000013,"[2.36735, 2.36905]",2.36760,2.36835,6.128200e+05,2.36800,0.000000,2.33640


In [3]:
# get top n coins

def get_top_n_coins(n=60):
    top_n_coins_df = df.sort_values(by='dayNtlVlm', ascending=False).head(n)
    top_n_coins_df = top_n_coins_df.reset_index(drop=True)
    top_n_coins_df.head()

    # get first coin name
    top_n_coins = top_n_coins_df.get('name')
    first_coin = top_n_coins[0]
    print("top 1st coin:", first_coin)
    return top_n_coins

top_n_coins = get_top_n_coins(2)

top 1st coin: BTC


In [4]:
%time
import time
from hyperliquid.info import Info
from hyperliquid.utils import constants as hl_constants
import utils.now_timestamp as now_timestamp 
import models.candles as candles
import importlib
importlib.reload(candles)
importlib.reload(now_timestamp)
info = Info(hl_constants.MAINNET_API_URL, skip_ws=True)

def fetch_coins_by_names(names):
    start = now_timestamp.get_timestamp_from_today_midnight_utc(days=-30)
    end = now_timestamp.get_timestamp_from_today_midnight_utc(days=0)
    error_coins = []
    for coin in names:
        try:
            time.sleep(0.2)
            print(coin)
            result = info.candles_snapshot(coin, '15m', start, end)
            _candles = candles.CandleList.from_response(result)
            _candles.to_csv()
        except Exception as e:
            print(f"coin {coin}: not fetched")
            print(e)
            error_coins.append(coin)
            continue

fetch_coins_by_names(top_n_coins)

CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 8.11 µs
BTC
ETH
